In [1]:
import xarray as xr
import numpy as np
import os
import arrow
import pandas as pd
import metpy.calc as mpcalc
from metpy.units import units
import glob
import re

In [2]:
latlon_pair_surface = [('110.0', '20.0'),
                        ('110.0', '20.125'),('110.125', '20.125'),('110.25', '20.125'),('110.375', '20.125'),('110.5', '20.125'),
                        ('110.0', '20.25'),('110.125', '20.25'),('110.25', '20.25'),('110.375', '20.25'),('110.5', '20.25')]

latlon_pair_pressure = [('110.0', '20.0'),('110.25', '20.0'),('110.5', '20.0'),
                         ('110.0', '20.25'),('110.25', '20.25'),('110.5', '20.25')]
station_file_list = ['59754.20200101-20221031.csv','59757.20200101-20221031.csv', '59758.20200101-20221031.csv']
if os.environ['COMPUTERNAME'] == 'DESKTOP-EQAO3M5':
  computer_flag = 'home'
else:
  computer_flag = 'office'

if computer_flag == 'home':
  file_dir = "F:/github/pythonScript/seafog/"
else:
  file_dir = "H:/github/python/seafog/"

fcHour_list = np.array(range(0, 240+1, 1))# list(range(0, 72+1, 3)) + list(range(78, 240+1, 6))
# fcHour_list = list(range(0, 72+1, 3)) + list(range(78, 168+1, 6))
single_level_name_list = ['visi','v100','v10m','u100','u10m','t2mm','t2md','sstk']
multi_level_name_list = ['vwnd','uwnd','temp','rhum']
levels_selected = [1000.0, 950.0, 925.0, 900.0, 850.0]

multi_level_columns = []
for multi_name_key in multi_level_name_list:
    for iLevel in levels_selected:
        multi_level_columns.append(f'{multi_name_key}{int(iLevel)}')

dir_path = os.path.normpath(os.path.join(file_dir, './data/CFdata/concat/201508-202205/'))
input_path =  os.path.normpath(os.path.join(file_dir, './data/netcdf/CFdata/fullhour/'))

In [13]:

dirPath = input_path.replace('\\','/')
reg =f"{dirPath}/*.nc"
path_list = glob.glob(reg)
surface_files_groups = []
for lonlat in latlon_pair_surface:
    surface_files = []
    for iPath in path_list:
        (filepath,filename) = os.path.split(iPath)
        matched_Group = re.match(r'.*?lon(.*?)_lat(.*?)\.(\S{4})\.nc', filename)
        lon_str = matched_Group.group(1)
        lat_str = matched_Group.group(2)
        varname_str = matched_Group.group(3)
        if varname_str in single_level_name_list and lon_str==lonlat[0] and lat_str==lonlat[1]:
            file_info = {'varname':varname_str, 'lon':lon_str, 'lat':lat_str, 'file':filename, 'dirpath':filepath}
            surface_files.append(file_info)
    if len(surface_files) == len(single_level_name_list):
        surface_files_groups.append(surface_files)

pressure_files_groups = []
for lonlat in latlon_pair_pressure:
    pressure_files = []
    for iPath in path_list:
        (filepath,filename) = os.path.split(iPath)
        matched_Group = re.match(r'.*?lon(.*?)_lat(.*?)\.(\S{4})\.nc', filename)
        lon_str = matched_Group.group(1)
        lat_str = matched_Group.group(2)
        varname_str = matched_Group.group(3)
        if varname_str in multi_level_name_list and lon_str==lonlat[0] and lat_str==lonlat[1]:
            file_info = {'varname':varname_str, 'lon':lon_str, 'lat':lat_str, 'file':filename, 'dirpath':filepath}
            pressure_files.append(file_info)
    if len(pressure_files) == len(multi_level_name_list):
        pressure_files_groups.append(pressure_files)

